# ARTIFICIAL NEURAL NETWORK

## CHURN MODELLING
A churn model is a mathematical representation of how churn impacts your business. Churn calculations are built on existing data (the number of customers who left your service during a given time period). A predictive churn model extrapolates on this data to show future potential churn rates.

###IMPORT LIBRARIES AND DATASETS

In [ ]:
# Code to read csv file into colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
# Get the file
file = drive.CreateFile({'id':'1Wd5EAMPU01Gix8DOxXJ7XX7x44pfCuGq'}) # replace the id with id of file you want to access
file.GetContentFile('Churn_Modelling.csv') 


In [ ]:
# Importing Libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Importing the Dateset, Creating a DataFrame and Checking the Total Number of Records/Observations and Attributes/Features
dataset = pd.read_csv('Churn_Modelling.csv')
shape = dataset.shape   # Returns tuple with number of rows and columns

In [ ]:
print(shape)

(10000, 14)


In [ ]:
import tensorflow as tf

In [ ]:
tf.__version__

'2.5.0'

In [ ]:
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [ ]:
X = dataset.iloc[:, 3:-1].values #RowNumber, CustomerId, Surname do not have any significance and do not add any value
y = dataset.iloc[:, -1].values #Selecting only the last column

## Part 1 - Data Preprocessing

### Encoding Categorical Variable

### Label Encoding Gender Column

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

In [ ]:
X

array([[619, 'France', 0, ..., 1, 1, 101348.88],
       [608, 'Spain', 0, ..., 0, 1, 112542.58],
       [502, 'France', 0, ..., 1, 0, 113931.57],
       ...,
       [709, 'France', 0, ..., 0, 1, 42085.58],
       [772, 'Germany', 1, ..., 1, 0, 92888.52],
       [792, 'France', 0, ..., 1, 0, 38190.78]], dtype=object)

### OneHotEncoding "Geography" Column

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [ ]:
X

array([[1.0, 0.0, 0.0, ..., 1, 1, 101348.88],
       [0.0, 0.0, 1.0, ..., 0, 1, 112542.58],
       [1.0, 0.0, 0.0, ..., 1, 0, 113931.57],
       ...,
       [1.0, 0.0, 0.0, ..., 0, 1, 42085.58],
       [0.0, 1.0, 0.0, ..., 1, 0, 92888.52],
       [1.0, 0.0, 0.0, ..., 1, 0, 38190.78]], dtype=object)

### Splitting data into train and test set

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

### Feature Scaling

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Part 2 - Building an ANN

### Initializing the ANN

In [ ]:
ann = tf.keras.models.Sequential()

### Adding Input & First Hidden Layer

In [ ]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

### Adding Second Layer

In [ ]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

### Adding an Output Layer

In [ ]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the ANN

###Compiling the ANN

In [ ]:
ann.compile(optimizer='adam', loss='binary_crossentropy' , metrics=['accuracy'])

###Training the ANN on the Training set

In [ ]:
ann.fit(X_train, y_train, batch_size=32, epochs=100)

Epoch 1/100
250/250 [==============================] - 1s 1ms/step - loss: 0.5813 - accuracy: 0.7520
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4867 - accuracy: 0.7969
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4454 - accuracy: 0.8049
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4229 - accuracy: 0.8163
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4117 - accuracy: 0.8254
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4021 - accuracy: 0.8314
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3925 - accuracy: 0.8374
Epoch 8/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3823 - accuracy: 0.8439
Epoch 9/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3729 - accuracy: 0.8500
Epoch 10/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3643 - accura

##Part 4 - Making the Prediction and Evaluating the Model

###Predicting the result

In [ ]:
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]]))>0.5)

[[False]]


###Predicting the Test set result

In [ ]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred), 1), y_test.reshape(len(y_test), 1)), 1))

[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


###Making the Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1515   80]
 [ 200  205]]


0.86